In [1]:
# !pip install cebra
# from google.colab import drive
# drive.mount('/content/drive/')

In [2]:
# !unzip /content/drive/MyDrive/Cebra_data/2020_11_9_MV1_run_brain.zip

In [3]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from cebra import CEBRA
from PIL import Image
import cv2
import torch
import pickle

In [4]:
with open('feature_labels', 'rb') as f:
    behav_feature = pickle.load(f)

In [5]:
from logging import exception
def process_brain(path):
  try:
    brain_seq = cv2.imreadmulti(path)[1]
    brain_seq = np.array(brain_seq)
    brain_mask = (np.sum(brain_seq, axis=0) > 0)
    flat_seq = (brain_seq[:, brain_mask])
    return flat_seq.astype(float)
  except:
    print(np.shape(brain_seq))

In [6]:
# iterate through all brain data and process it
#https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
import os
neural_data = []
name_data = []
behav_data = []

filepath = "2020_11_9_MV1_run_brain"
for iter, file in enumerate(os.listdir(filepath)):
     filename = os.fsdecode(file)
     if filename.endswith(".tif"):
         neural_data.append(process_brain(filepath + '/' + filename))
         name_data.append(filename)
         split_name = filename.split('_')
         session_data = []
         session_data.append(int(split_name[2].split('.')[0]))
         session_data.append(int(split_name[1]))
         behav_data.append(session_data)
        # for using low ram systems
        #  if iter > 50:
        #     break
         continue
     else:
         continue

In [ ]:
# split into training and validation set

In [7]:
# neural_data_pred = []
# name_data_pred = []
# move_nomove_pred = []

# filepath = "2020_12_4_MV1_run_brain"
# for iter, file in enumerate(os.listdir(filepath)):
#      filename = os.fsdecode(file)
#      if filename.endswith(".tif"):
#          neural_data_pred.append(process_brain(filepath + '/' + filename))
#          name_data_pred.append(filename)
#          split_name = filename.split('_')
#          if split_name[0] == 'move':
#           move_nomove_pred.append(np.ones_like(neural_data_pred[-1]))
#          else:
#           move_nomove_pred.append(np.zeros_like(neural_data_pred[-1]))
#         #  if iter > 100:
#         #     break
#          continue
#      else:
#          continue

FileNotFoundError: [Errno 2] No such file or directory: '2020_12_4_MV1_run_brain'

In [ ]:
multi_cebra_model = CEBRA.load('/content/drive/MyDrive/Cebra_data/cebra_multi_model.pt')

In [ ]:
image_data = []
filepath = "2020_12_4_MV1_run_behavior"
for file in os.listdir(filepath):
     filename = os.fsdecode(file)
     if filename.endswith(".tif"):
         image_data.append(cv2.imreadmulti(filepath + '/' + filename)[1])
         continue
     else:
         continue

In [ ]:

import cebra
fig = plt.figure()
move_nomove = []
ax1 = plt.subplot(121, projection='3d')
ax2 = plt.subplot(122, projection='3d')
index = 20
for run, data in enumerate(zip(neural_data, name_data)):
  is_move = True
  if data[1].split('_')[0] == 'move':
    is_move = True
    move_nomove.append(np.ones_like(data[0][:, 1]))
  else:
    is_move = False
    move_nomove.append(np.zeros_like(data[0][:, 1]))
  try:
    embedding = multi_cebra_model.transform(data[0], session_id=run)
  except:
    print(run)
    move_nomove.pop()
  if is_move:
    ax1 = cebra.plot_embedding(embedding, idx_order=(5,6, 7), ax=ax1, title='moving')
  else:
    ax2 = cebra.plot_embedding(embedding, ax=ax2, title = 'no move')

plt.show()

In [ ]:
move_embedding_train = []
for run, data in enumerate(neural_data):
  try:
    move_embedding_train.append(multi_cebra_model.transform(data, session_id=run))
  except:
    print(run)

In [ ]:
move_embedding_pred = []
for run, data in enumerate(neural_data_pred):
  try:
    move_embedding_pred.append(multi_cebra_model.transform(data, session_id=run))
  except:
    print(run)

In [ ]:
move_embeddings_flat = move_embedding_train[0]
move_move_nomove_flat = move_nomove[0]
for data in zip(move_embedding_train[1::], move_nomove[1::]):
    move_embeddings_flat = np.concatenate((move_embeddings_flat, data[0]))
    move_move_nomove_flat = np.concatenate((move_move_nomove_flat, data[1]))

In [ ]:
move_embeddings_flat_pred = move_embedding_pred[0]
move_move_nomove_flat_pred = move_nomove[0]
for data in zip(move_embedding_train[1::], move_nomove[1::]):
    move_embeddings_flat_pred = np.concatenate((move_embeddings_flat, data[0]))
    move_move_nomove_flat_pred = np.concatenate((move_move_nomove_flat, data[1]))

In [ ]:
print(np.shape(move_embeddings_flat))
print(np.shape(move_move_nomove_flat))
print(np.shape(move_embeddings_flat_pred))
print(np.shape(move_move_nomove_flat_pred))

In [ ]:
import sklearn.metrics
move_decoder = cebra.KNNDecoder(n_neighbors=36, metric="cosine")
move_decoder.fit(move_embeddings_flat, move_move_nomove_flat)
move_pred = move_decoder.predict(move_embeddings_flat_pred)
test_score = sklearn.metrics.r2_score(move_move_nomove_flat_pred, move_pred)


In [ ]:
test_score

In [ ]:
print(np.sum(move_move_nomove_flat) / np.size(move_move_nomove_flat))

In [ ]:
print(np.sum(np.ones_like(move_pred) * ((np.array(move_pred) * np.array(move_move_nomove_flat_pred)) > 0.5)) / np.sum(move_move_nomove_flat_pred))


In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.subplot(111)

ax = cebra.plot_loss(multi_cebra_model, color='deepskyblue', label='loss', ax=ax)
plt.show()